In [1]:
!wget https://www.gutenberg.org/files/11/11-0.txt -O alice.txt

--2025-05-11 20:41:11--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151191 (148K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 147.65K   606KB/s    in 0.2s    

2025-05-11 20:41:16 (606 KB/s) - ‘alice.txt’ saved [151191/151191]



In [2]:
with open("./alice.txt") as f:
  alice_in_wonderland = f.read()

In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# Usa un modello di embedding gratuito (senza chiave)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Inizializza il semantic chunker con quel modello
semantic_chunker = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile"
)


/tmp/ipykernel_1208904/1400814468.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/storage/data_4T_b/andreacutuli/miniconda3/envs/retrievers/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
semantic_chunks = semantic_chunker.create_documents([alice_in_wonderland])

In [5]:
for semantic_chunk in semantic_chunks:
  if "waistcoat-pocket_" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))

Alice’s Evidence




CHAPTER I. Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her. There was nothing so _very_ remarkable in that; nor did Alice think it
so _very_ much out of the way to hear the Rabbit say to itself, “Oh
dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the
time it all seemed quite natural); but when the Rabbit actually _took 

In [8]:
pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

semantic_chunk_vectorstore = FAISS.from_documents(semantic_chunks, embedding=embedding_model)

In [10]:
semantic_chunk_retriever = semantic_chunk_vectorstore.as_retriever(search_kwargs={"k" : 1})

In [11]:
semantic_chunk_retriever.invoke("Who has a pocket watch?")

[Document(id='e58114c9-86aa-4dec-8b25-be79e3c2c138', metadata={}, page_content='“What day of the month\nis it?” he said, turning to Alice: he had taken his watch out of his\npocket, and was looking at it uneasily, shaking it every now and then,\nand holding it to his ear. Alice considered a little, and then said “The fourth.”\n\n“Two days wrong!” sighed the Hatter. “I told you butter wouldn’t suit\nthe works!” he added looking angrily at the March Hare. “It was the _best_ butter,” the March Hare meekly replied. “Yes, but some crumbs must have got in as well,” the Hatter grumbled:\n“you shouldn’t have put it in with the bread-knife.”\n\nThe March Hare took the watch and looked at it gloomily: then he dipped\nit into his cup of tea, and looked at it again: but he could think of\nnothing better to say than his first remark, “It was the _best_ butter,\nyou know.”\n\nAlice had been looking over his shoulder with some curiosity. “What a\nfunny watch!” she remarked. “It tells the day of the m

In [12]:
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [13]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-beta",
    max_new_tokens=512,
    temperature=0.7
)
base_model = HuggingFacePipeline(pipeline=pipe)


Fetching 8 files:   0%|          | 0/8 [07:49<?, ?it/s]
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 